# Analyse des temps de paroles des élections europeenes

Preparations des fichiers pour affichage.<br>
<br>
Le site de l'arcom nous fournie plusieurs fichiers à l'adresse suivante : <br>
<a href = "https://www.arcom.fr/actualites/elections-europeennes-2024-publication-des-releves-des-temps-de-parole"> page arcom </a><br>


In [1]:
# import des bibliotheques
import pandas as pd
import os


In [2]:
#lister les nom des fichiers dans les données brutes
liste_fichier =os.listdir('donnee_brut')


In [3]:
# faire les trucs pour tout les fichiers :
temps_parole_EU2024 = pd.DataFrame()
for element in liste_fichier:
    # lire le fichier 
    tempo_df = pd.read_csv('donnee_brut/'+element,encoding='latin',sep=';',header=3)
    # garder les 2 derniere ligne du temps presidentiel
    df_president = tempo_df.tail(2)
    # supprimer les ligne du temps presidentiel
    tempo_df = tempo_df[:-2]
    # rajouter une ligne pour le temps presidentiel 
    tempo_df.loc[len(tempo_df)]=['temps_president',df_president.iat[1,1]]
    # rajouter le nom de la chaine
    tempo_df['Nom du media'] = (element[30:-4])
    # mettre le temps total du media en colonne
    #total = tempo_df.loc[tempo_df['Listes candidates']=='Total des temps de parole','Total période (durée)'].to_string()
    #tempo_df['temps total'] = total[-8:]
    tempo_df = tempo_df[tempo_df['Listes candidates']!='Total des temps de parole']
    # concat les dataframe entre eux
    temps_parole_EU2024 = pd.concat([temps_parole_EU2024,tempo_df])
    

In [4]:
couleur_pol = pd.read_csv('donnee_contruite/couleurs.csv',encoding='latin')

Liste des medias contenante les fichiers :<br>
Canal plus n'a pas fourni de temps de paroles<br>

In [5]:
print(temps_parole_EU2024['Nom du media'].unique())

['BFMBusiness' 'BFMTV' 'C8' 'Cnews' 'Euronews' 'Europe1' 'France2'
 'France24' 'France3' 'France5' 'FranceCulture' 'FranceInfo' 'FranceInter'
 'LCI' 'M6' 'RFI' 'RMC-Story' 'RMC' 'RMCDecouverte' 'RTL' 'RadioClassique'
 'SudRadio' 'TF1' 'TMC' 'TV5Monde' 'franceinfo_']


In [6]:
# listez les media en fonction de leur categories
liste_tv_gen = ['TMC', 'France2', 'France3','M6', 'RMC-Story',
       'TF1', 'France5', 'C8', 'RMCDecouverte', 'TV5Monde',
       ]

liste_tv_info = ['LCI', 'franceinfo_',
       'BFMTV', 'France24', 'Euronews', 'Cnews']

liste_radio = ['RTL', 'RFI', 'FranceInfo', 'FranceCulture',
       'SudRadio', 'RMC', 'Europe1', 'BFMBusiness',
       'RadioClassique', 'FranceInter']

# ajouter les categories dans le dataframe :
temps_parole_EU2024['Type de media'] = 'inconnu'

for media in liste_tv_gen: 
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'TV généraliste'
for media in liste_tv_info:
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'TV info continue' 
for media in liste_radio:
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'radio' 

Catégorisation des médias : <br>

In [7]:
temps_parole_EU2024['Type de media'].unique()

array(['radio', 'TV info continue', 'TV généraliste'], dtype=object)

In [8]:
temps_parole_EU2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 711 entries, 0 to 40
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Listes candidates      711 non-null    object
 1   Total période (durée)  711 non-null    object
 2   Nom du media           711 non-null    object
 3   Type de media          711 non-null    object
dtypes: object(4)
memory usage: 27.8+ KB


In [9]:
# voir pour changer le type de données de la periode tout convertir en seconde puis en heure
temps_parole_EU2024['heure'] = temps_parole_EU2024['Total période (durée)'].str[:2]
temps_parole_EU2024['minute'] = temps_parole_EU2024['Total période (durée)'].str[3:5]
temps_parole_EU2024['seconde'] = temps_parole_EU2024['Total période (durée)'].str[-2:]

temps_parole_EU2024['heure'] = temps_parole_EU2024['heure'].astype(int)
temps_parole_EU2024['minute'] = temps_parole_EU2024['minute'].astype(int)
temps_parole_EU2024['seconde'] = temps_parole_EU2024['seconde'].astype(int)

temps_parole_EU2024['Durée en seconde'] = temps_parole_EU2024['heure']*3600 + temps_parole_EU2024['minute']*60 +temps_parole_EU2024['seconde']
temps_parole_EU2024['Durée en minute'] = temps_parole_EU2024['Durée en seconde']/60 

In [10]:
temps_parole_EU2024.drop(columns=['heure','minute','seconde'],inplace=True)

# Temps de parole en secondes des listes :<br>

In [11]:
temps_parole_EU2024.groupby(by='Listes candidates').agg({'Durée en seconde':'sum'}).reset_index()

,Listes candidates,Durée en seconde
0,ALLIANCE CENTRISTE ET TERRITOIRES EN MOUVEMENT,194
1,ALLIANCE RURALE,9592
2,BESOIN D?EUROPE,184834
3,CHANGER L?EUROPE,7712
4,DEFENDRE LES ENFANTS,775
5,DEMOCRATIE REPRESENTATIVE,461
6,ECOLOGIE POSITIVE ET TERRITOIRES,1369
7,ESPERANTO LANGUE COMMUNE,949
8,EUROPE ECOLOGIE,79731
9,EUROPE EQUITABLE,489


In [12]:
#calcul du temps total :

df_temps_total = temps_parole_EU2024.groupby(by='Nom du media').agg({'Durée en seconde':'sum'}).reset_index().rename(columns={'Durée en seconde':'Temps total'})



In [13]:
temps_parole_EU2024 = pd.merge(temps_parole_EU2024,df_temps_total,on='Nom du media')
temps_parole_EU2024 = pd.merge(temps_parole_EU2024,couleur_pol,on='Listes candidates')

In [14]:
temps_parole_EU2024.to_csv('donnee_contruite/temps_paroles_EU2024.csv',index=False)

In [15]:
temps_parole_EU2024

,Listes candidates,Total période (durée),Nom du media,Type de media,Durée en seconde,Durée en minute,Temps total,Couleur
0,ALLIANCE RURALE,00:20:23,BFMTV,TV info continue,1223,20.383333,159919,Autres
1,BESOIN D?EUROPE,07:55:17,BFMTV,TV info continue,28517,475.283333,159919,Centriste et Liberaux
2,CHANGER L?EUROPE,00:19:42,BFMTV,TV info continue,1182,19.700000,159919,Sociaux Democrate
3,DEFENDRE LES ENFANTS,00:02:05,BFMTV,TV info continue,125,2.083333,159919,Autres
4,DEMOCRATIE REPRESENTATIVE,00:02:00,BFMTV,TV info continue,120,2.000000,159919,Autres
...,...,...,...,...,...,...,...,...
667,"POUR UN MONDE SANS FRONTIERES NI PATRONS, URGE...",00:00:43,franceinfo_,TV info continue,43,0.716667,128677,gauche radicale
668,POUR UNE AUTRE EUROPE,00:01:34,franceinfo_,TV info continue,94,1.566667,128677,Autres
669,POUR UNE DEMOCRATIE REELLE: DECIDONS NOUS-MEMES!,00:02:22,franceinfo_,TV info continue,142,2.366667,128677,Autres
670,POUR UNE HUMANITE SOUVERAINE,00:00:21,franceinfo_,TV info continue,21,0.350000,128677,Autres
